# Reranking
* The reranking stage happens after the ranking stage
* The ranking stage learns the user's preference relation over items
  * Does the user prefer item A over item B?
* The reranking stage learns the user's preference relation over recommendation lists
  * Does the user prefer recommendation list A over recommendation list B? 
* The reranking stage is modelled as a binary quadratic programming problem
  * Given a set of items, find the optimal subset by solving a QP
  * We get a ranked list by iteratively taking subsets

In [ ]:
using JuMP
import Ipopt

In [ ]:
@memoize function get_similarity_metric(medium::String, weights = nothing)
    alphas = ["$medium/all/WatchSimilarity", "$medium/all/GenreSimilarity"]
    if medium == "anime"
        push!(alphas, "$medium/all/TagSimilarity")
    end
    if isnothing(weights)
        weights = ones(Float32, length(alphas))
    end
    convert.(Float32, weights ./ sum(weights))
    sum([read_params(x)["S"] for x in alphas] .* weights)
end

In [ ]:
function solve_quadratic_program(
    list_size,
    similarity_metric,
    relevance_scores,
    constraint_spec,
)
    # solves the quadratic program:
    # minimize x' * similarity_metric * x - relevance_scores * x 
    # with the constraints:
    # sum(x) = list_size
    # x \in {0, 1} are binary variables    
    # attributes[i]' * x <= constraints[i] for all i

    # scale problem by list_size
    N = size(similarity_metric)[1]
    similarity_metric = similarity_metric ./ list_size^2
    relevance_scores = relevance_scores ./ list_size
    attributes = constraint_spec[1]
    constraints = [f(list_size) for f in constraint_spec[2]]

    # make the initial optimization problem convex
    # this will not change the solution to the minimization problem
    posdef_penalty = 0
    while !LinearAlgebra.isposdef(similarity_metric)
        similarity_metric -= posdef_penalty * LinearAlgebra.I(N)
        posdef_penalty = max(1, posdef_penalty * 2)
        similarity_metric += posdef_penalty * LinearAlgebra.I(N)
        if posdef_penalty > 2^16
            @assert false "similarity matrix is not positive definite"
        end
    end

    # solve the mixed-integer quadratic problem by
    # iteratively solving the QP with an increasing penalty 
    # for non-binary solutions
    nonbinary_penalty = 0
    model = nothing
    solution = zeros(N)
    epsilon = 0.01
    while sum(solution .> epsilon) != list_size
        # model
        model = Model(
            optimizer_with_attributes(Ipopt.Optimizer, "print_level" => 0, "sb" => "yes"),
        )
        @variable(model, 0 <= x[1:N] <= 1)

        # constraints
        @constraint(model, sum(x) == list_size)
        for i = 1:length(constraints)
            @constraint(model, attributes[i]' * x <= constraints[i])
        end

        #objective
        @expression(model, relevance, relevance_scores' * x)
        @expression(model, similarity, x' * similarity_metric * x)
        @expression(model, penalty, nonbinary_penalty * x' * (1 .- x))
        @objective(model, Min, similarity - relevance + penalty)

        # solve
        optimize!(model)
        y = value.(x)
        nonbinary_penalty = max(1, nonbinary_penalty * 2)
        if nonbinary_penalty > 2^10
            @info "could not solve miqp $list_size. dropping constraint"
            @assert length(constraints) > 0
            skip = (list_size % length(constraints)) + 1
            return solve_quadratic_program(
                list_size,
                similarity_metric,
                relevance_scores,
                (
                    [v for (i, v) in Iterators.enumerate(constraint_spec[1]) if i != skip],
                    [v for (i, v) in Iterators.enumerate(constraint_spec[2]) if i != skip],
                ),
            )
        end
        solution = value.(x)
    end

    collect(1:length(solution))[solution.>epsilon]
end;

In [ ]:
function get_intrarelated_constraints(df, medium, limitfrac)
    S = read_params("$medium/all/RelatedSeries/similarity_matrix")["S"]
    attributes = []
    constraints = []
    for uid in df.uid
        x = S[:, uid][df.uid]
        if sum(x) > 1
            if any(all(x .== y) for y in attributes)
                continue
            end
            push!(attributes, collect(x))
            push!(constraints, N -> 1 + limitfrac * N)
        end
    end
    attributes, constraints
end

function get_interrelated_constraints(df, limitfrac)
    attributes = [convert.(Float32, df.is_related .!= 0)]
    constraints = [N -> ceil(limitfrac * N)]
    attributes, constraints
end
function get_crossrelated_constraints(df, limitfrac)
    attributes = [convert.(Float32, df.is_cross_related .!= 0)]
    constraints = [N -> ceil(limitfrac * N)]
    attributes, constraints
end

function get_ptw_constraints(df, limitfrac)
    attributes = [convert.(Float32, df.ptw .!= 0)]
    constraints = [N -> ceil(limitfrac * N)]
    attributes, constraints
end

function merge_constraints(args...)
    Tuple(reduce(vcat, [x[t] for x in args]) for t = 1:2)
end

function get_constraint_spec(df, medium)
    merge_constraints(
        get_intrarelated_constraints(df, medium, 0.01),
        get_interrelated_constraints(df, 0.1),
        get_crossrelated_constraints(df, 0.1),
        get_ptw_constraints(df, 0.1),
    )
end;

In [ ]:
function rerank(df, medium::String, list_size, similarity_spec)
    GC.gc()
    S =
        similarity_spec[2] *
        get_similarity_metric(medium, similarity_spec[1])[df.uid.+1, df.uid.+1]
    constraint_spec = get_constraint_spec(df, medium)
    candidates = solve_quadratic_program(list_size, S, df.score, constraint_spec)

    # TODO block reranking
    # first select items in groups of 5
    # then rerank is 5 group
    order = []
    while length(candidates) > 0
        new_S = S[candidates, candidates]
        new_score = df.score[candidates]
        new_constraint_spec =
            ([x[candidates] for x in constraint_spec[1]], constraint_spec[2])
        new_order = solve_quadratic_program(
            length(candidates) - 1,
            new_S,
            new_score,
            new_constraint_spec,
        )
        worst_candidate = candidates[findfirst(x -> x ∉ new_order, 1:length(candidates))]
        candidates = candidates[new_order]
        pushfirst!(order, worst_candidate)
    end
    
    score = sum(df.score[order]) / length(order) - sum(S[order, order]) / length(order)^2
    df[order, :]
end
rerank(args...) = x -> rerank(x, args...);